In [1]:
#imports
import pandas as pd
import numpy as np
import re
import pprint
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk import ne_chunk, pos_tag
nltk.download('words')
from nltk.corpus import words as nltk_words
import os
import wget
import pickle
from allennlp.predictors.predictor import Predictor
from allennlp.models.archival import load_archive
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Y\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Y\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
data_path =  r'C:\Users\Y\Documents\MILA\Gender_Generalization\Data\tfidf.csv'
data = pd.read_csv(data_path)
#print(data[:10])
data.astype('str')
sentences = data['Sentences']
test = sentences.astype('str')
print(test[0])
print(type(sentences[0]))
print(type(test[1]))
label = data["Label"]

" They encounter Haber, wheelchair bound, on their way to lunch, and Haber recognizes Orr, but cannot come out of his catatonia . "
<class 'str'>
<class 'str'>


In [3]:
def preprocess(data):
    clean_sentences = []
    for s in data:
        #new_s = re.sub(r'\n+', ' ', data[i])                   # remove breakline
        new_s = re.sub(r'[^\w\/,:;.?\\\']', ' ', s)  # removes all weird characters except punctuations
        new_s = new_s.replace("//", "")                  # remove quotation marks
        new_s = re.sub(r'\d+.', '', new_s)               # removes number followed by a .
        new_s = re.sub(r'/\s\s+/g', ' ', new_s)
        new_s = new_s.replace('/', '')                   # removes forward slash
        new_s = new_s.replace('_', '')
        new_s = re.sub(r' +', ' ', new_s)                # removes extra spaces
        new_s = new_s.lstrip()                           # removes spaces at the beginning of a string
        new_s = new_s.rstrip()                           # removes spaces at the end of a string
        wordcount = len(new_s.split())
        if new_s != '' and wordcount > 5:                # sentences < 5 words are rejected
            clean_sentences.append(new_s)
        else:
            clean_sentences.append(np.nan)

    return clean_sentences

In [4]:
clean = preprocess(test)


In [8]:
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer

path = './tfidf.csv'
token_dict = {}

#data_path =  r'C:\Users\Y\Documents\MILA\Gender_Generalization\Data\tfidf.csv'
#df = pd.read_csv(data_path)
#sent = df['Sentences']

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

for s in range(0, len(test)):
    tokenize(test[s])

token_dict = test #.translate(None, string.punctuation)

tfidf = TfidfVectorizer(tokenizer=tokenize) #, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values())

str = 'all great and precious things are lonely.'
response = tfidf.transform([str])
print(response)

feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print(feature_names[col], ' - ', response[0, col])

TypeError: 'numpy.ndarray' object is not callable

In [9]:
def vocab_sentence(data):
    vocabulary = set()
    for sent in data:
        vocabulary.update(sent.split())
    return vocabulary
        

In [10]:
vocab = vocab_sentence(test)

In [11]:
print(vocab)

{'husband', 'awaits', 'questions', 'worried', 'affords.', 'Nomura', 'contractus', 'deserved,', 'corporations', 'cases', 'in', 'merchant,', 'Issues', 'LAW', 'killed', 'motel', 'auspices.', 'assign', 'choicest', 'carries', "auditor's", 'distinctly', 'party,', 'Caesar', 'dummy', 'Girl', 'riches', 'window', 'served', 'dressed,', 'chances', 'unchallenged', 'Fund', 'indulge', 'station', 'farrier.', 'turned', 'passport', 'grip,', 'objective', 'Althea', 'duty', 'warranty,', 'near', 'influence.', 'extended', "I'm", 'manor.', 'B.', 'nights,', 'hears', 'm', 'Crutcher', 'draws', 'Inconvenient', 'box,', 'Calvert', 'ratified', 'partly', 'Fortnightly,', 'Cooper', 'who,', 'hatched.', 'California', 'worried,', 'Blair', 'Jackson', 'woman', 'file', 'light,', 'amnesty', 'crowding', 'baking,', 'bitterly', 'comply', 'IV', 'upon;', 'Honour.', 'assistants', 'going', 'unless', 'certiorari', 'them.', 'Frowning,', 'calculation', 'lives', 'adjudge', 'bomb', 'loaded.', 'Chul', 'taken', 'fearful', 'eye,', 'door.', 

In [12]:
vocab_good = list(vocab)


In [13]:
word_index = {w: id for id, w in enumerate(vocab_good)}

In [14]:
print(word_index)

{'husband': 0, 'awaits': 1, 'questions': 2, 'worried': 3, 'affords.': 4, 'Nomura': 5, 'contractus': 6, 'deserved,': 7, 'corporations': 8, 'cases': 9, 'in': 10, 'merchant,': 11, 'Issues': 12, 'LAW': 13, 'killed': 14, 'motel': 15, 'auspices.': 16, 'assign': 17, 'choicest': 18, 'carries': 19, "auditor's": 20, 'distinctly': 21, 'party,': 22, 'Caesar': 23, 'dummy': 24, 'Girl': 25, 'riches': 26, 'window': 27, 'served': 28, 'dressed,': 29, 'chances': 30, 'unchallenged': 31, 'Fund': 32, 'indulge': 33, 'station': 34, 'farrier.': 35, 'turned': 36, 'passport': 37, 'grip,': 38, 'objective': 39, 'Althea': 40, 'duty': 41, 'warranty,': 42, 'near': 43, 'influence.': 44, 'extended': 45, "I'm": 46, 'manor.': 47, 'B.': 48, 'nights,': 49, 'hears': 50, 'm': 51, 'Crutcher': 52, 'draws': 53, 'Inconvenient': 54, 'box,': 55, 'Calvert': 56, 'ratified': 57, 'partly': 58, 'Fortnightly,': 59, 'Cooper': 60, 'who,': 61, 'hatched.': 62, 'California': 63, 'worried,': 64, 'Blair': 65, 'Jackson': 66, 'woman': 67, 'file'

In [15]:
tfidf = TfidfVectorizer(vocabulary = vocab_good)

In [16]:
tfidf.fit(test)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=['husband', 'awaits', 'questions', 'worried', 'affords.', 'Nomura', 'contractus', 'deserved,', 'corporations', 'cases', 'in', 'merchant,', 'Issues', 'LAW', 'killed', 'motel', 'auspices.', 'assign', 'choicest', 'carries', "auditor's", 'distinctly', 'party,', 'Caesar', 'dummy', 'Girl', 'ric..., 'slain', 'fraternity.', 'Seth', 'reconciles', 'summer', 'Parliament', 'men,', 'chasing', 'surely'])

In [17]:
tfidf.transform(test)

<1982x11199 sparse matrix of type '<class 'numpy.float64'>'
	with 41369 stored elements in Compressed Sparse Row format>

In [20]:
import operator 
for sent in test:
    score = {}
    print(sent)
    print()
    X = tfidf.transform([sent])
    for word in sent.split():
        score[word] = X[0,tfidf.vocabulary_[word]]
    sortedscore = sorted(score.items(), key=operator.itemgetter(1), reverse = True)
    print(sortedscore)

" They encounter Haber, wheelchair bound, on their way to lunch, and Haber recognizes Orr, but cannot come out of his catatonia . "

[('encounter', 0.37696320615348106), ('wheelchair', 0.37696320615348106), ('catatonia', 0.37696320615348106), ('recognizes', 0.3576137707038792), ('come', 0.2545597711075727), ('cannot', 0.24773077280059602), ('way', 0.22630615911821697), ('their', 0.2162705300165846), ('out', 0.19530325896862352), ('but', 0.16091765825736046), ('on', 0.13674706363811692), ('his', 0.0886559322620587), ('and', 0.08421603223920972), ('of', 0.08199367354378316), ('to', 0.07498406694413089), ('"', 0.0), ('They', 0.0), ('Haber,', 0.0), ('bound,', 0.0), ('lunch,', 0.0), ('Haber', 0.0), ('Orr,', 0.0), ('.', 0.0)]
" Cha duk's wife, Yeong ja becomes involved with Nicolai, who rejects her proposal to run away with him. "

[('ja', 0.38869530627895543), ('rejects', 0.3687436648040426), ('proposal', 0.34360756924915975), ('becomes', 0.31468446694677044), ('involved', 0.314684466946770

[('bomber', 0.29509353937243393), ('mission', 0.2799464552579257), ('badly', 0.2691994280476319), ('damaged', 0.2691994280476319), ('burning', 0.2691994280476319), ('base', 0.2608633861200371), ('pilot', 0.25405234393312376), ('leader', 0.2433053167228299), ('fly', 0.2433053167228299), ('trying', 0.23890525981861555), ('over', 0.17266824078716594), ('back', 0.16995014732512817), ('after', 0.1601993739814734), ('to', 0.11739773723207324), ('is', 0.08823364875441439), ('in', 0.07245461075675722), ('his', 0.06940144929402432), ('and', 0.0659258161531368), ('On', 0.0), ('May', 0.0), ('Squadron', 0.0), ('Peter', 0.0), ('Carter', 0.0), ('a', 0.0), ('Royal', 0.0), ('Air', 0.0), ('Force', 0.0), ('Lancaster', 0.0), ('England', 0.0), ('Germany.', 0.0)]
Colonel Morgan learns from his Indian scout Bookbinder that Baker is planning something but is thwarted at learning details.

[('planning', 0.3822113395786944), ('thwarted', 0.3822113395786944), ('learns', 0.337875727364323), ('learning', 0.337875

" Julie's mother doesn't want the union either, as it will be an inter faith marriage, and she wants to return to England . "

[('inter', 0.37671471366672643), ('union', 0.3169702951263468), ('faith', 0.3169702951263468), ('wants', 0.3106021667494193), ('return', 0.2839140216676932), ('want', 0.28062374322390293), ('mother', 0.2543919664265916), ('will', 0.18920547038872024), ('she', 0.1578143578654021), ('an', 0.15757288837843955), ('to', 0.14986927555430365), ('be', 0.1337931462271575), ('it', 0.13306725516905296), ('as', 0.12840416379002534), ('and', 0.08416051740133132), ('the', 0.06103442021178488), ('"', 0.0), ("Julie's", 0.0), ("doesn't", 0.0), ('either,', 0.0), ('marriage,', 0.0), ('England', 0.0), ('.', 0.0)]
" Once home, Bean admires his room, decorated with photographs of the Langley family, before taking a final look at the ruined, original Whistler's Mother he smuggled back with him, and then going to bed . "

[('admires', 0.2903858348291676), ('photographs', 0.29038583482

[('revered', 0.538093341031652), ('rich', 0.5104731324855866), ('society', 0.45275535816532003), ('son', 0.39231242781340114), ('by', 0.18607472043612774), ('is', 0.16089114980506156), ('his', 0.1265512549087981), ('and', 0.12021355245364496), ('"', 0.0), ('Senthil', 0.0), ('Nathan', 0.0), ('a', 0.0), ('businessman,', 0.0), ('Raja', 0.0), ('.', 0.0)]
" Having now observed Honey 's many qualities beyond his minor eccentricities, and after becoming very close to Elspeth, she decides to make the arrangement permanent by marrying the scientist . "

[('qualities', 0.2804341133090511), ('arrangement', 0.2804341133090511), ('marrying', 0.2804341133090511), ('permanent', 0.2660394941930132), ('scientist', 0.2660394941930132), ('minor', 0.2558263493954808), ('becoming', 0.2558263493954808), ('decides', 0.24790441883934713), ('observed', 0.22329665492577686), ('beyond', 0.22329665492577686), ('close', 0.21991300067605446), ('many', 0.1842942718961687), ('now', 0.17882169169575413), ('very', 0.17

[('murdered', 0.4631767222244164), ('grudge', 0.4631767222244164), ('executioner', 0.4631767222244164), ('held', 0.29369757173471733), ('against', 0.2627757138204385), ('who', 0.22820368731740373), ('would', 0.2170705877522195), ('her', 0.1732107128683319), ('be', 0.16450079775862064), ('by', 0.16016826919878316), ('is', 0.13849090802490055), ('"', 0.0), ('Pierrepoint', 0.0), ('Mary,', 0.0), ('a', 0.0), ('.', 0.0)]
He realizes that Victoria has fallen in love with the letters and is concerned that she will be disappointed by the real Roger.

[('fallen', 0.365889817586527), ('disappointed', 0.365889817586527), ('realizes', 0.34710877664807904), ('letters', 0.34710877664807904), ('concerned', 0.3337834160389163), ('real', 0.29622133416202046), ('love', 0.26161055264237454), ('that', 0.20846152811245214), ('will', 0.18376865180834726), ('has', 0.15351528721043078), ('she', 0.15327956280197458), ('be', 0.12994860060255203), ('with', 0.1292435680216571), ('by', 0.12652608818259653), ('the',

...

[('to', 0.26618813662099877), ('common', 0.22930727124370234), ('of', 0.1819198403141434), ('denotes', 0.1672740915337324), ('occupations', 0.1672740915337324), ('establish', 0.1672740915337324), ('worship', 0.1672740915337324), ('privileges', 0.1672740915337324), ('orderly', 0.1672740915337324), ('bodily', 0.15868795054254786), ('engage', 0.15868795054254786), ('acquire', 0.15868795054254786), ('dictates', 0.15868795054254786), ('restraint', 0.15259598656016685), ('recognized', 0.15259598656016685), ('happiness', 0.15259598656016685), ('enjoy', 0.14787069218947016), ('essential', 0.14787069218947016), ('useful', 0.1440098455689823), ('merely', 0.14074554626328048), ('freedom', 0.1379178815866013), ('pursuit', 0.1379178815866013), ('generally', 0.13542370457779773), ('individual', 0.12933174059541674), ('according', 0.12933174059541674), ('bring', 0.12763675387607165), ('free', 0.12763675387607165), ('those', 0.11378918787164234), ('long', 0.11215945861304762), ('home', 0.1092339830037

[('court', 0.32740493456063263), ('mandate', 0.30983915755339075), ('ignored', 0.29393518421866316), ('usurpation', 0.29393518421866316), ('announced', 0.2826511355603215), ('governor', 0.2554631135672523), ('determined', 0.2554631135672523), ('the', 0.2509970098607458), ('meet', 0.22365516689779721), ('once', 0.2157840546223609), ('again', 0.20632470819706827), ('such', 0.17152110593655415), ('would', 0.1452080055227387), ('with', 0.10944474623307782), ('by', 0.1071435571222228), ('that', 0.08826365363034498), ('he', 0.08040509907055969), ('and', 0.06922008316870576), ('of', 0.06739344933619304), ('The', 0.0), ('State', 0.0), ('Supreme', 0.0), ('Court', 0.0), ('resistance.', 0.0)]
Thus, it has been held that a suit against the Secretary of the Treasury to review his decision on the rate of duty to be exacted on imported sugar would disturb the whole revenue system of the Government and would in effect be a suit against the United States.

[('suit', 0.35470611814978537), ('against', 0.

[('shall', 0.544618056662573), ('elect', 0.30376820426131235), ('act', 0.21693245778814274), ('have', 0.20151030139825393), ('until', 0.1183089980922459), ('or', 0.11115302755899276), ('the', 0.10638406788766848), ('then', 0.09719788969426237), ('who', 0.09243197667102954), ('declaring', 0.08557181506623912), ('wherein', 0.08292199428695299), ('chosen', 0.0807569330657836), ('beginning', 0.0807569330657836), ('accordingly', 0.07594205106532809), ('if', 0.07564811211162423), ('fixed', 0.07355909455371167), ('failed', 0.07355909455371167), ('provide', 0.07252583791948002), ('neither', 0.07252583791948002), ('manner', 0.07157533395566613), ('as', 0.06394591880776983), ('nor', 0.06334423880936255), ('for', 0.0616527515463239), ('case', 0.05264743448102475), ('such', 0.05192751951292717), ('before', 0.05060671952848414), ('time', 0.049418232446580175), ('person', 0.049138808727556035), ('law', 0.049001524422257546), ('may', 0.04508418374857704), ('been', 0.04423222773746923), ('and', 0.0419

[('nothing', 0.4722208195187126), ('give', 0.424571013088208), ('duty', 0.41714962255978183), ('can', 0.33445467903310727), ('which', 0.25874496498821437), ('from', 0.2570159505999781), ('him', 0.2333623334459527), ('is', 0.19560368656078062), ('This', 0.0), ('a', 0.0), ('dispensation.', 0.0)]
Where the gravity of the crime and other factors such as the age and education of the defendant, the conduct of the court or the prosecuting officials, and the complicated nature of the offense charged and the possible defenses thereto render criminal proceedings without counsel so apt to result in injustice as to be fundamentally unfair, the latter group majority holds that the accused must have legal assistance under the amendment whether he pleads guilty or elects to stand trial, whether he requests counsel or not.

[('the', 0.32405870375852713), ('counsel', 0.22212156573868838), ('whether', 0.20490994181347558), ('or', 0.18231528548736778), ('gravity', 0.15385729683223592), ('complicated', 0.

[('affectionate', 0.25305063257581184), ('depositing', 0.24006160128421933), ('surrounded', 0.24006160128421933), ('circle', 0.24006160128421933), ('constantly', 0.23084573691911708), ('devoted', 0.23084573691911708), ('intelligent', 0.22369735716318467), ('of', 0.2201652640764217), ('citizens', 0.21291850511690905), ('aid', 0.19843921010919477), ('eggs', 0.19565180997082984), ('her', 0.1892628811298589), ('seen', 0.18093074103928897), ('by', 0.1750117391248658), ('bee', 0.16629853455820268), ('queen', 0.16524811609758389), ('most', 0.16423104994903281), ('the', 0.16399464192050486), ('such', 0.14008405102853622), ('my', 0.13610218699359722), ('some', 0.13369844285213345), ('in', 0.12426354791788005), ('have', 0.10872223286853269), ('an', 0.10584646055592548), ('and', 0.05653315730363406), ('By', 0.0), ('hives,', 0.0), ('Philadelphia', 0.0), ('Apiary,', 0.0), ('cells,', 0.0), ('children.', 0.0)]
It should enable the Apiarian, when he has removed the combs from a common hive, to place t